# Pré-processamento dos dados

> Neste notebook iremos realizar o pré-processamento dos dados a serem utilizados na criação do modelo preditivo

In [1]:
# Importando bibliotecas necessárias
import snowflake.connector
import pandas as pd
import numpy as np
import os
from dadosfera import *
import seaborn as sns
import matplotlib.pyplot as plt

### 1. Recuperar dados do passo anterior

In [2]:
# Recuperando dados do passo anterior
data = dadosfera_utils.get_inputs() 
df = data['df']

print(f'Colunas atuais: {df.columns}')
df.head()

Colunas atuais: Index(['__SDC_ROW', '__SDC_SHEET_ID', '__SDC_SPREADSHEET_ID', 'away_score',
       'away_team', 'city', 'country', 'date', 'home_score', 'home_team',
       'neutral', 'tournament', 'total_points_home', 'previous_points_home',
       'rank_home', 'rank_change_home', 'total_points_away',
       'previous_points_away', 'rank_away', 'rank_change_away'],
      dtype='object')


,__SDC_ROW,__SDC_SHEET_ID,__SDC_SPREADSHEET_ID,away_score,away_team,city,country,date,home_score,home_team,neutral,tournament,total_points_home,previous_points_home,rank_home,rank_change_home,total_points_away,previous_points_away,rank_away,rank_change_away
0,17699,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,1.0,Mali,Libreville,Gabon,1993-01-01,1.0,Ghana,True,Friendly,34.0,0.0,39.0,0.0,22.0,0.0,69.0,0.0
1,17700,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,1.0,Burkina Faso,Libreville,Gabon,1993-01-02,1.0,Gabon,False,Friendly,27.0,0.0,55.0,0.0,11.0,0.0,97.0,0.0
2,17702,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,0.0,Mali,Libreville,Gabon,1993-01-03,1.0,Burkina Faso,True,Friendly,11.0,0.0,97.0,0.0,22.0,0.0,69.0,0.0
3,17703,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,3.0,Ghana,Libreville,Gabon,1993-01-03,2.0,Gabon,False,Friendly,27.0,0.0,55.0,0.0,34.0,0.0,39.0,0.0
4,17704,701885009,1OWDpobAEf2132MRWgkgkQEbzT1fhP1r1XBLiCup_R_s,1.0,Tanzania,Kampala,Uganda,1993-01-08,3.0,Uganda,False,Friendly,12.0,0.0,92.0,0.0,15.0,0.0,80.0,0.0


### 2. Corrigir tipagem

In [3]:
df["home_score"] = df["home_score"].astype(float)
df["away_score"] = df["away_score"].astype(float)

### 3. Adicionar novas colunas

In [4]:
# Calculando os resultados de acordo com a vantagem de estar jogando em casa
def calculate_result(home, away):
    if home > away:
        return pd.Series([0, 3, 0])
    if home < away:
        return pd.Series([1, 0, 3])
    else: # em caso de empate
        return pd.Series([2, 1, 1])

results = df.apply(lambda x: calculate_result(x["home_score"], x["away_score"]), axis=1)
df[["result", "home_team_points", "away_team_points"]] = results

In [5]:
# Calculando a diferença de ranking entre os dois times adversários
df["rank_dif"] = df["rank_home"] - df["rank_away"]

In [6]:
# Calculando a diferença de pontuação entre os dois times adversários
df["sg"] = df["home_score"] - df["away_score"]

In [7]:
# Calculando um valor envolvendo a os pontos e a posição no ranking
df["points_home_by_rank"] = df["home_team_points"]/df["rank_away"]
df["points_away_by_rank"] = df["away_team_points"]/df["rank_home"]

In [8]:
# Adicionando ao dataset estatísticas dos últimos 5 jogos
home_team = df[["date", "home_team", "home_score", "away_score", "rank_home", "rank_away","rank_change_home", "total_points_home", "result", "rank_dif", "points_home_by_rank", "home_team_points"]]
away_team = df[["date", "away_team", "away_score", "home_score", "rank_away", "rank_home","rank_change_away", "total_points_away", "result", "rank_dif", "points_away_by_rank", "away_team_points"]]

home_team.columns = [h.replace("home_", "").replace("_home", "").replace("away_", "suf_").replace("_away", "_suf") for h in home_team.columns]
away_team.columns = [a.replace("away_", "").replace("_away", "").replace("home_", "suf_").replace("_home", "_suf") for a in away_team.columns]

team_stats = home_team.append(away_team)
team_stats_raw = team_stats.copy()

In [9]:
stats_val = []

for index, row in team_stats.iterrows():
    team = row["team"]
    date = row["date"]
    past_games = team_stats.loc[(team_stats["team"] == team) & (team_stats["date"] < date)].sort_values(by=['date'], ascending=False)
    last5 = past_games.head(5)
    
    goals = past_games["score"].mean()
    goals_l5 = last5["score"].mean()
    
    goals_suf = past_games["suf_score"].mean()
    goals_suf_l5 = last5["suf_score"].mean()
    
    rank = past_games["rank_suf"].mean()
    rank_l5 = last5["rank_suf"].mean()
    
    if len(last5) > 0:
        points = past_games["total_points"].values[0] - past_games["total_points"].values[-1]#qtd de pontos ganhos
        points_l5 = last5["total_points"].values[0] - last5["total_points"].values[-1] 
    else:
        points = 0
        points_l5 = 0
        
    gp = past_games["team_points"].mean()
    gp_l5 = last5["team_points"].mean()
    
    gp_rank = past_games["points_by_rank"].mean()
    gp_rank_l5 = last5["points_by_rank"].mean()
    
    stats_val.append([goals, goals_l5, goals_suf, goals_suf_l5, rank, rank_l5, points, points_l5, gp, gp_l5, gp_rank, gp_rank_l5])

In [10]:
stats_cols = ["goals_mean", "goals_mean_l5", "goals_suf_mean", "goals_suf_mean_l5", "rank_mean", "rank_mean_l5", "points_mean", "points_mean_l5", "game_points_mean", "game_points_mean_l5", "game_points_rank_mean", "game_points_rank_mean_l5"]

stats_df = pd.DataFrame(stats_val, columns=stats_cols)

full_df = pd.concat([team_stats.reset_index(drop=True), stats_df], axis=1, ignore_index=False)

home_team_stats = full_df.iloc[:int(full_df.shape[0]/2),:]
away_team_stats = full_df.iloc[int(full_df.shape[0]/2):,:]

home_team_stats = home_team_stats[home_team_stats.columns[-12:]]
away_team_stats = away_team_stats[away_team_stats.columns[-12:]]

home_team_stats.columns = ['home_'+str(col) for col in home_team_stats.columns]
away_team_stats.columns = ['away_'+str(col) for col in away_team_stats.columns]

match_stats = pd.concat([home_team_stats, away_team_stats.reset_index(drop=True)], axis=1, ignore_index=False)
full_df = pd.concat([df, match_stats.reset_index(drop=True)], axis=1, ignore_index=False)

In [11]:
# Criando uma coluna com a importância do jogo por meio do tipo de competição
def find_friendly_tournaments(x):
    if x == "Friendly":
        return 1
    else: return 0

full_df["is_friendly"] = full_df["tournament"].apply(lambda x: find_friendly_tournaments(x)) 
full_df = pd.get_dummies(full_df, columns=["is_friendly"])

In [12]:
base_df = full_df[["date", "home_team", "away_team", "rank_home", "rank_away","home_score", "away_score","result", "rank_dif", "rank_change_home", "rank_change_away", 'home_goals_mean',
       'home_goals_mean_l5', 'home_goals_suf_mean', 'home_goals_suf_mean_l5',
       'home_rank_mean', 'home_rank_mean_l5', 'home_points_mean',
       'home_points_mean_l5', 'away_goals_mean', 'away_goals_mean_l5',
       'away_goals_suf_mean', 'away_goals_suf_mean_l5', 'away_rank_mean',
       'away_rank_mean_l5', 'away_points_mean', 'away_points_mean_l5','home_game_points_mean', 'home_game_points_mean_l5',
       'home_game_points_rank_mean', 'home_game_points_rank_mean_l5','away_game_points_mean',
       'away_game_points_mean_l5', 'away_game_points_rank_mean',
       'away_game_points_rank_mean_l5',
       'is_friendly_0', 'is_friendly_1']]

base_df.head()

,date,home_team,away_team,rank_home,rank_away,home_score,away_score,result,rank_dif,rank_change_home,...,home_game_points_mean,home_game_points_mean_l5,home_game_points_rank_mean,home_game_points_rank_mean_l5,away_game_points_mean,away_game_points_mean_l5,away_game_points_rank_mean,away_game_points_rank_mean_l5,is_friendly_0,is_friendly_1
0,1993-01-01,Ghana,Mali,39.0,69.0,1.0,1.0,2,-30.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,1993-01-02,Gabon,Burkina Faso,55.0,97.0,1.0,1.0,2,-42.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
2,1993-01-03,Burkina Faso,Mali,97.0,69.0,1.0,0.0,0,28.0,0.0,...,1.0,1.0,0.018182,0.018182,1.0,1.0,0.025641,0.025641,0,1
3,1993-01-03,Gabon,Ghana,55.0,39.0,2.0,3.0,1,16.0,0.0,...,1.0,1.0,0.010309,0.010309,1.0,1.0,0.014493,0.014493,0,1
4,1993-01-08,Uganda,Tanzania,92.0,80.0,3.0,1.0,0,12.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1


In [13]:
base_df.isna().sum()

date                               0
home_team                          0
away_team                          0
rank_home                          0
rank_away                          0
home_score                         0
away_score                         0
result                             0
rank_dif                           0
rank_change_home                   0
rank_change_away                   0
home_goals_mean                   94
home_goals_mean_l5                94
home_goals_suf_mean               94
home_goals_suf_mean_l5            94
home_rank_mean                    94
home_rank_mean_l5                 94
home_points_mean                   0
home_points_mean_l5                0
away_goals_mean                  107
away_goals_mean_l5               107
away_goals_suf_mean              107
away_goals_suf_mean_l5           107
away_rank_mean                   107
away_rank_mean_l5                107
away_points_mean                   0
away_points_mean_l5                0
h

In [14]:
# Excluindo valores nulos do dataset
base_df_no_fg = base_df.dropna()
df = base_df_no_fg

In [15]:
def no_draw(x):
    if x == 2:
        return 1
    else:
        return x
    
df["target"] = df["result"].apply(lambda x: no_draw(x))

/opt/conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [16]:
def create_db(df):
    columns = ["home_team", "away_team", "target", "rank_dif", "home_goals_mean", "home_rank_mean", "away_goals_mean", "away_rank_mean", "home_rank_mean_l5", "away_rank_mean_l5", "home_goals_suf_mean", "away_goals_suf_mean", "home_goals_mean_l5", "away_goals_mean_l5", "home_goals_suf_mean_l5", "away_goals_suf_mean_l5", "home_game_points_rank_mean", "home_game_points_rank_mean_l5", "away_game_points_rank_mean", "away_game_points_rank_mean_l5","is_friendly_0", "is_friendly_1"]
    
    base = df.loc[:, columns]
    base.loc[:, "goals_dif"] = base["home_goals_mean"] - base["away_goals_mean"]
    base.loc[:, "goals_dif_l5"] = base["home_goals_mean_l5"] - base["away_goals_mean_l5"]
    base.loc[:, "goals_suf_dif"] = base["home_goals_suf_mean"] - base["away_goals_suf_mean"]
    base.loc[:, "goals_suf_dif_l5"] = base["home_goals_suf_mean_l5"] - base["away_goals_suf_mean_l5"]
    base.loc[:, "goals_per_ranking_dif"] = (base["home_goals_mean"] / base["home_rank_mean"]) - (base["away_goals_mean"] / base["away_rank_mean"])
    base.loc[:, "dif_rank_agst"] = base["home_rank_mean"] - base["away_rank_mean"]
    base.loc[:, "dif_rank_agst_l5"] = base["home_rank_mean_l5"] - base["away_rank_mean_l5"]
    base.loc[:, "dif_points_rank"] = base["home_game_points_rank_mean"] - base["away_game_points_rank_mean"]
    base.loc[:, "dif_points_rank_l5"] = base["home_game_points_rank_mean_l5"] - base["away_game_points_rank_mean_l5"]
    
    model_df = base[["home_team", "away_team", "target", "rank_dif", "goals_dif", "goals_dif_l5", "goals_suf_dif", "goals_suf_dif_l5", "goals_per_ranking_dif", "dif_rank_agst", "dif_rank_agst_l5", "dif_points_rank", "dif_points_rank_l5", "is_friendly_0", "is_friendly_1"]]
    return model_df

In [17]:
model_db = create_db(df)

model_db

,home_team,away_team,target,rank_dif,goals_dif,goals_dif_l5,goals_suf_dif,goals_suf_dif_l5,goals_per_ranking_dif,dif_rank_agst,dif_rank_agst_l5,dif_points_rank,dif_points_rank_l5,is_friendly_0,is_friendly_1
2,Burkina Faso,Mali,0,28.0,0.000000,0.0,0.000000,0.0,-0.007459,16.000000,16.0,-0.007459,-0.007459,0,1
3,Gabon,Ghana,1,16.0,0.000000,0.0,0.000000,0.0,-0.004183,28.000000,28.0,-0.004183,-0.004183,0,1
10,Uganda,Tanzania,0,12.0,2.000000,2.0,-2.000000,-2.0,0.026630,-12.000000,-12.0,0.037500,0.037500,0,1
18,Togo,Zimbabwe,1,47.0,-1.000000,-1.0,-1.000000,-1.0,-0.009804,-23.000000,-23.0,0.002854,0.002854,1,0
23,Cameroon,Finland,1,-22.0,0.000000,0.0,0.000000,0.0,0.000000,-39.000000,-39.0,0.002622,0.002622,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22679,Norway,Serbia,1,11.0,-0.021864,-0.8,-0.098214,0.0,-0.002203,2.999458,-3.4,0.032883,0.016348,1,0
22680,Sweden,Slovenia,1,-45.0,0.273962,0.2,-0.220077,0.2,0.010679,-10.729778,10.2,0.030313,-0.020314,1,0
22681,Kosovo,Cyprus,0,-2.0,0.280376,0.6,-0.380645,0.0,0.001040,11.803495,1.6,-0.002177,-0.002827,1,0
22682,Greece,Northern Ireland,0,-9.0,0.281395,0.2,-0.300394,-1.2,0.005758,-1.784947,0.6,0.026101,0.018570,1,0


### 4. Exportar o dataset criado para o próximo passo

In [18]:
dict_df = {'model_db':model_db, 'team_stats_raw':team_stats_raw}
dadosfera_utils.output(dict_df, name="dict_df")